# Pct of N Sources at Watershed and Country level

#### Dependencies


In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

#### File Paths and Files

In [2]:
DATA_IN = '/home/cascade/projects/wastewater/data/interim/N_effluent_output/'
DATA_OUT = '/home/cascade/projects/wastewater/data/interim/N_effluent_output/'

In [ ]:
## Open N pour pointfiles

allWaste_fn = 'effluent_N_pourpoints.shp'
openWaste_fn = 'effluent_N_open_pourpoints.shp'
sepWaste_fn = 'effluent_N_septic_pourpoints.shp'
treatedWaste_fn = 'effluent_N_treated_pourpoints.shp'

allWaste = gpd.read_file(DATA_IN+allWaste_fn) # all
openWaste = gpd.read_file(DATA_IN+openWaste_fn) # open Def
sepWaste = gpd.read_file(DATA_IN+sepWaste_fn) # septic tanks
treatedWaste = gpd.read_file(DATA_IN+treatedWaste_fn) # treated

waste_list = [allWaste, openWaste, sepWaste, treatedWaste] # have a list for later

In [3]:
## Open N country shape files
allWaste_fn = 'effluent_N_countries.shp'
openWaste_fn = 'effluent_N_open_countries.shp'
sepWaste_fn = 'effluent_N_septic_countries.shp'
treatedWaste_fn = 'effluent_N_treated_countries.shp'

allWaste = gpd.read_file(DATA_IN+allWaste_fn) # all
openWaste = gpd.read_file(DATA_IN+openWaste_fn) # open Def
sepWaste = gpd.read_file(DATA_IN+sepWaste_fn) # septic tanks
treatedWaste = gpd.read_file(DATA_IN+treatedWaste_fn) # treated

waste_list = [allWaste, openWaste, sepWaste, treatedWaste] # have a list for later

In [4]:
allWaste.head()

,ISO3,poly_type,count,effluent,geometry
0,ABW,GADM,183,2.320971e+07,POLYGON ((-6910816.274421509 1537164.105192429...
1,AFG,GADM,645513,5.128715e+10,"POLYGON ((6211413.408736277 3848509.713654268,..."
2,AGO,GADM,1255033,1.238465e+10,(POLYGON ((1159542.671178129 -2115445.01835647...
3,AIA,GADM,84,1.018501e+07,(POLYGON ((-6134902.918835666 2230758.12862136...
4,ALA,GADM,88,1.074733e+06,(POLYGON ((1389515.950805801 6852829.269928327...


#### Let's look at the data

In [5]:
# check len
for waste in waste_list:
    print(len(waste))

539
539
539
539


In [6]:
allWaste.head()

,ISO3,poly_type,count,effluent,geometry
0,ABW,GADM,183,2.320971e+07,POLYGON ((-6910816.274421509 1537164.105192429...
1,AFG,GADM,645513,5.128715e+10,"POLYGON ((6211413.408736277 3848509.713654268,..."
2,AGO,GADM,1255033,1.238465e+10,(POLYGON ((1159542.671178129 -2115445.01835647...
3,AIA,GADM,84,1.018501e+07,(POLYGON ((-6134902.918835666 2230758.12862136...
4,ALA,GADM,88,1.074733e+06,(POLYGON ((1389515.950805801 6852829.269928327...


In [7]:
# rename columns
allWaste.rename(columns = {'effluent':'effluent_all'}, inplace = True) 
openWaste.rename(columns = {'effluent':'effluent_open'}, inplace = True) 
sepWaste.rename(columns = {'effluent':'effluent_sep'}, inplace = True) 
treatedWaste.rename(columns = {'effluent':'effluent_treated'}, inplace = True)

In [8]:
# For countries we'll need to make ids to merge on
ids = list(range(0,len(allWaste)))
allWaste['ids'] = ids
openWaste['ids'] = ids
sepWaste['ids'] = ids
treatedWaste['ids'] = ids

In [9]:
allWaste.head()

,ISO3,poly_type,count,effluent_all,geometry,ids
0,ABW,GADM,183,2.320971e+07,POLYGON ((-6910816.274421509 1537164.105192429...,0
1,AFG,GADM,645513,5.128715e+10,"POLYGON ((6211413.408736277 3848509.713654268,...",1
2,AGO,GADM,1255033,1.238465e+10,(POLYGON ((1159542.671178129 -2115445.01835647...,2
3,AIA,GADM,84,1.018501e+07,(POLYGON ((-6134902.918835666 2230758.12862136...,3
4,ALA,GADM,88,1.074733e+06,(POLYGON ((1389515.950805801 6852829.269928327...,4


In [10]:
# Try an inner join for now
df = pd.DataFrame()
df = allWaste.iloc[:,[0,3,5]]# for watershed allWaste.iloc[:,:2]

merge_list = waste_list[1:] # short list

geog = 'ids'

for i, waste in enumerate(merge_list):
    df = df.merge(waste.iloc[:,[3,5]], on = geog , how = 'inner') #update cols for countries vs watershed

In [11]:
df.head()

,ISO3,effluent_all,ids,effluent_open,effluent_sep,effluent_treated
0,ABW,2.320971e+07,0,8.014450e+06,9.193381e+05,1.427593e+07
1,AFG,5.128715e+10,1,5.050395e+10,1.979618e+08,5.852399e+08
2,AGO,1.238465e+10,2,1.124436e+10,5.900537e+08,5.502364e+08
3,AIA,1.018501e+07,3,7.468404e+06,9.041301e+05,1.812475e+06
4,ALA,1.074733e+06,4,2.886590e+05,1.586590e+03,7.844878e+05


In [12]:
# check sums
df['sum'] = df['effluent_open'] + df['effluent_sep'] + df['effluent_treated']
df['pct_total'] = df['sum'] / df['effluent_all']

#### Make Pct

In [13]:
df.head()

,ISO3,effluent_all,ids,effluent_open,effluent_sep,effluent_treated,sum,pct_total
0,ABW,2.320971e+07,0,8.014450e+06,9.193381e+05,1.427593e+07,2.320971e+07,1.0
1,AFG,5.128715e+10,1,5.050395e+10,1.979618e+08,5.852399e+08,5.128715e+10,1.0
2,AGO,1.238465e+10,2,1.124436e+10,5.900537e+08,5.502364e+08,1.238465e+10,1.0
3,AIA,1.018501e+07,3,7.468404e+06,9.041301e+05,1.812475e+06,1.018501e+07,1.0
4,ALA,1.074733e+06,4,2.886590e+05,1.586590e+03,7.844878e+05,1.074733e+06,1.0


In [14]:
df['pct_open'] = df['effluent_open'] / df['effluent_all'] * 100
df['pct_sep'] = df['effluent_sep'] / df['effluent_all'] * 100
df['pct_treated'] = df['effluent_treated'] / df['effluent_all'] * 100

## Countries

In [15]:
## Add country names
names_fn = '/home/cascade/projects/wastewater/data/interim/country_codes.csv'
names = pd.read_csv(names_fn)
names.shape

cols = names[['iso3c', 'country.name.en']]
cols.rename(columns = {'iso3c':'ISO3'}, inplace = True)

/home/cascade/miniconda3/envs/geo/lib/python3.6/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [19]:
df_out = df.merge(cols, on = 'ISO3', how = 'left')

In [17]:
df_out.head()

,ISO3,effluent_all,ids,effluent_open,effluent_sep,effluent_treated,sum,pct_total,pct_open,pct_sep,pct_treated,country.name.en
0,ABW,2.320971e+07,0,8.014450e+06,9.193381e+05,1.427593e+07,2.320971e+07,1.0,34.530582,3.961006,61.508411,Aruba
1,AFG,5.128715e+10,1,5.050395e+10,1.979618e+08,5.852399e+08,5.128715e+10,1.0,98.472909,0.385987,1.141104,Afghanistan
2,AGO,1.238465e+10,2,1.124436e+10,5.900537e+08,5.502364e+08,1.238465e+10,1.0,90.792714,4.764396,4.442890,Angola
3,AIA,1.018501e+07,3,7.468404e+06,9.041301e+05,1.812475e+06,1.018501e+07,1.0,73.327413,8.877067,17.795520,Anguilla
4,ALA,1.074733e+06,4,2.886590e+05,1.586590e+03,7.844878e+05,1.074733e+06,1.0,26.858660,0.147626,72.993712,Åland Islands


In [18]:
## Save out as a .csv
fn_out = 'countries_N_Alldata.csv'
df_out.to_csv(DATA_OUT+fn_out)

In [20]:
## Add geometry for shape file
cols = allWaste[['ids', 'geometry']]
df_out_shape = df_out.merge(cols, on = 'ids', how = 'left')

In [21]:
df_out_shape.head()

,ISO3,effluent_all,ids,effluent_open,effluent_sep,effluent_treated,sum,pct_total,pct_open,pct_sep,pct_treated,country.name.en,geometry
0,ABW,2.320971e+07,0,8.014450e+06,9.193381e+05,1.427593e+07,2.320971e+07,1.0,34.530582,3.961006,61.508411,Aruba,POLYGON ((-6910816.274421509 1537164.105192429...
1,AFG,5.128715e+10,1,5.050395e+10,1.979618e+08,5.852399e+08,5.128715e+10,1.0,98.472909,0.385987,1.141104,Afghanistan,"POLYGON ((6211413.408736277 3848509.713654268,..."
2,AGO,1.238465e+10,2,1.124436e+10,5.900537e+08,5.502364e+08,1.238465e+10,1.0,90.792714,4.764396,4.442890,Angola,(POLYGON ((1159542.671178129 -2115445.01835647...
3,AIA,1.018501e+07,3,7.468404e+06,9.041301e+05,1.812475e+06,1.018501e+07,1.0,73.327413,8.877067,17.795520,Anguilla,(POLYGON ((-6134902.918835666 2230758.12862136...
4,ALA,1.074733e+06,4,2.886590e+05,1.586590e+03,7.844878e+05,1.074733e+06,1.0,26.858660,0.147626,72.993712,Åland Islands,(POLYGON ((1389515.950805801 6852829.269928327...


In [22]:
shp_out = gpd.GeoDataFrame(df_out_shape)

In [23]:
## Save out as a .csv
fn_out = 'countries_N_Alldata.shp'
shp_out.to_file(DATA_OUT+fn_out)

In [24]:
print('done')

done


## Watershed

#### Divide By Watershed size

In [ ]:
df.head()

In [ ]:
area = allWaste[['basin_id', 'area']]
df = df.merge(area, on = 'basin_id', how = 'left')
df.head()

In [ ]:
df.head()

In [ ]:
# Divide by area (km2 I think)
df['all_area'] = df['effluent_all'] / df['area']
df['open_area'] = df['effluent_open'] / df['area']
df['sep_area'] = df['effluent_sep'] / df['area']
df['treated_area'] = df['effluent_treated'] / df['area']

#### Add Geography

In [ ]:
allWaste.head()

In [ ]:
#### Add in pour point locations
geo = allWaste[['basin_id', 'geometry']]
pourpoints_out = df.merge(geo, on = 'basin_id', how = 'left')

In [ ]:
#### save out pourpoints as a csv
fn_out = 'pourpoints_N_Alldata.csv'
pourpoints_out.to_csv(DATA_OUT+fn_out)

In [ ]:
#### save out pourpoints as a shp file
fn_out = 'pourpoints_N_Alldata.shp'
pourpoints_out_shape = gpd.GeoDataFrame(pourpoints_out)
pourpoints_out_shape.to_file(DATA_OUT+fn_out)

In [ ]:
#### Add in watershed geo
watershed = gpd.read_file(DATA_IN+'effluent_N_watersheds.shp')
geo = watershed[['basin_id', 'geometry']]
watersheds_out = df.merge(geo, on = 'basin_id', how = 'left')
watersheds_out

In [ ]:
#### save out pourpoints as a shpe file
fn_out = 'watersheds_N_Alldata.shp'
watersheds_out_shape = gpd.GeoDataFrame(watersheds_out)
watersheds_out_shape.to_file(DATA_OUT+fn_out)

In [ ]:
watersheds_out_shape.head(5)